Héctor Asorey de Pablos

# APRENDIZAJE AUTOMÁTICO II - TRABAJO 2

Para esta práctica se ha optado por utilizar aprendizaje por refuerzo para completar el nivel 1-1 del juego Super Mario Bros para la Nintendo Entertainment System, lanzado en 1885.

Para ello, se va a utilizar el entorno de Gym "gym_super_mario_bros", y se van a entrenar modelos DQNAgent, Double DQNAgent y PPO.

<hr>

### Importar las librerías necesarias

In [1]:
import os
import gym
import numpy as np
from gym_super_mario_bros.actions import RIGHT_ONLY
from nes_py.wrappers import JoypadSpace
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.agents.dqn import DQNAgent
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

2023-05-19 12:46:43.671594: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 12:46:45.749602: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/hector/anaconda3/envs/Mjolnir/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda-11/lib64
2023-05-19 12:46:45.749744: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /ho

<hr>

### Crear el entorno del juego

In [3]:
env = gym.make('SuperMarioBros-1-1-v0')
env = JoypadSpace(env, RIGHT_ONLY)

<hr>

## Modelos DQNAgent o DDQNAgent

<hr>

### Definir parámetros del entorno para los modelos

In [4]:
n_actions = len(RIGHT_ONLY)
height, width, n_channels = env.observation_space.shape

<hr>

### Construir modelo CNN

In [4]:
def build_cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=(5, height, width, n_channels)))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(n_actions, activation='linear'))
    return model

model = build_cnn_model()

<hr>

### Definir la memoria y la política de exploración del modelo

In [5]:
memory = SequentialMemory(limit=1000000, window_length=5)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=37000)

<hr>

### Construir el modelo DQNAgent o DDQNAgent

In [6]:
dqn = DQNAgent(model=model,
               nb_actions=n_actions,
               memory=memory,
               nb_steps_warmup=4000,#50000,
               target_model_update=2000,#10000,
               policy=policy,
               #Quitar la línea de abajo en caso de no querer Double DQN
               enable_double_dqn = True)

dqn.compile(optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4))

2023-05-19 10:34:55.801807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-19 10:34:56.124284: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-19 10:34:56.124976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-19 10:34:56.127521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


<hr>

### Entrenamiento del modelo

In [8]:
dqn.fit(env, nb_steps=33000, visualize=False, verbose=1)

Training for 33000 steps ...
Interval 1 (0 steps performed)


/home/hector/.local/lib/python3.7/site-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-05-19 10:35:23.817878: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_1/BiasAdd' id:189 op device:{requested: '', assigned: ''} def:{{{node dense_1/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_1/MatMul, dense_1/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-19 10:35:23.833960: W tensorflow/c/c_api.cc:291] Operation '{name:'total_2/Assign' id:499 op device:{requested: '', assigned: ''} def:{{{node total_2/Assign}} = AssignVariableOp[_has_manual_control_depende

   25/10000 [..............................] - ETA: 6:34 - reward: 0.6000done, took 6.774 seconds


<hr>

### Forma alternativa de entrenar el modelo

Se puede entrenar el modelo de forma distinta para saber el reward que obtiene el modelo a cada paso que da gracias a un callback

<hr>

### Definición del callback

In [11]:
class PrintRewardCallback(tf.keras.callbacks.Callback):
    def on_step_end(self, step, logs={}):
        if 'reward' in logs:
            print(f"Step {step}: reward = {logs['reward']}")

<hr>

### Entrenamiento del modelo

In [13]:
dqn.fit(env, nb_steps=42000, visualize=False, verbose=2, callbacks=[PrintRewardCallback()])

Training for 42000 steps ...
Step 0: reward = 0.0
Step 1: reward = 0.0
Step 2: reward = 0.0
Step 3: reward = 0.0
Step 4: reward = 0.0
Step 5: reward = 1.0
Step 6: reward = 0.0
Step 7: reward = 1.0
Step 8: reward = 0.0
Step 9: reward = 1.0
Step 10: reward = 1.0
Step 11: reward = 0.0
Step 12: reward = 1.0
Step 13: reward = 1.0
Step 14: reward = 1.0
Step 15: reward = 1.0
Step 16: reward = 1.0
Step 17: reward = 1.0
Step 18: reward = 0.0
Step 19: reward = 1.0
Step 20: reward = 2.0
Step 21: reward = 1.0
Step 22: reward = 1.0
Step 23: reward = 2.0
Step 24: reward = 1.0
Step 25: reward = 2.0
Step 26: reward = 1.0
Step 27: reward = 2.0
Step 28: reward = 1.0
Step 29: reward = 2.0
Step 30: reward = 2.0
Step 31: reward = 1.0
Step 32: reward = 2.0
Step 33: reward = 2.0
Step 34: reward = 2.0
Step 35: reward = 1.0
Step 36: reward = 2.0
Step 37: reward = 2.0
Step 38: reward = 1.0
Step 39: reward = 1.0
Step 40: reward = 2.0
Step 41: reward = 2.0
Step 42: reward = 2.0
Step 43: reward = 1.0
Step 44: rewa

<hr>

### Guardar el modelo

In [ ]:
model.save('mario_33_dqn.h5')

<hr>

### Probar el desempeño del modelo en el juego

In [6]:
from tensorflow.keras.models import load_model

modelToTest = load_model('mario_dqn_2000.h5')

# Define the memory buffer and the exploration policy
memory = SequentialMemory(limit=1000000, window_length=3)
policy = EpsGreedyQPolicy()

# Create the DQN agent
dqn = DQNAgent(model=modelToTest,
               nb_actions=n_actions,
               memory=memory,
               nb_steps_warmup=0,
               target_model_update=1000,
               policy=policy,
               enable_double_dqn = True)

# Compile the DQN agent
dqn.compile(optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4))

# Evaluate the agent for 10 episodes
dqn.test(env, nb_episodes=10, visualize=True)

2023-05-19 12:53:02.580588: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_4/kernel/Assign' id:1042 op device:{requested: '', assigned: ''} def:{{{node dense_4/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_4/kernel, dense_4/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-19 12:53:04.621355: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_5/bias/Assign' id:1294 op device:{requested: '', assigned: ''} def:{{{node dense_5/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_5/bias, dense_5/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger a

Testing for 10 episodes ...


2023-05-19 12:53:06.749367: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_1_3/BiasAdd' id:1076 op device:{requested: '', assigned: ''} def:{{{node dense_1_3/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_1_3/MatMul, dense_1_3/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-19 12:53:06.778013: W tensorflow/c/c_api.cc:291] Operation '{name:'total_6/Assign' id:1445 op device:{requested: '', assigned: ''} def:{{{node total_6/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](total_6, total_6/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after runni

Episode 1: reward: 1039.000, steps: 717


KeyboardInterrupt: 

<hr>

### Reentrenar el modelo donde lo dejó en caso de ser necesario

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('mario_33_dqn.h5')

# Define the memory buffer and the exploration policy
memory = SequentialMemory(limit=1000000, window_length=5)
policy = EpsGreedyQPolicy()

# Create the DQN agent
dqn = DQNAgent(model=model,
               nb_actions=n_actions,
               memory=memory,
               nb_steps_warmup=0,
               target_model_update=1000,
               policy=policy)

# Compile the DQN agent
dqn.compile(optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4))

# Evaluate the agent for 10 episodes
dqn.test(env, nb_episodes=10, visualize=True)

<hr>

## Modelo PPO

<hr> 

### Importar librerías necesarias

In [7]:
from stable_baselines3 import PPO

<hr>

### Creación del modelo

In [15]:
model2 = PPO('CnnPolicy', env, verbose=1, tensorboard_log='./logs/', learning_rate=0.000001, 
            n_steps=512) 

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


<hr>

### Creación de callbacks

Este callback guarda el modelo cada n pasos definidos

In [16]:
from stable_baselines3.common.callbacks import BaseCallback

CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}_v4'.format(self.n_calls))
            self.model.save(model_path)

        return True
    
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

<hr>

### Entrenamiento del modelo

In [17]:
model2.learn(total_timesteps=200000, callback = callback)

Logging to ./logs/PPO_14


/home/hector/anaconda3/envs/Mjolnir/lib/python3.7/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 139 |
|    iterations      | 1   |
|    time_elapsed    | 3   |
|    total_timesteps | 512 |
----------------------------


KeyboardInterrupt: 

<hr>

### Guardar el modelo

In [ ]:
model2.save('model_20000')

<hr>

### Cargar y probar el modelo entrenado en el juego

In [5]:
import stable_baselines3

In [8]:
 model3 = PPO.load('./train/best_model_100000_v4')

In [14]:
# Start the game 
state = env.reset()

# Loop through the game
while True: 
    # Make a copy of the state array
    state_copy = state.copy()

    # Predict the action
    action, _ = model3.predict(state_copy)

    # Take the action in the environment
    state, reward, done, info = env.step(action)

    env.render()


ValueError: cannot step in a done environment! call `reset`

<hr>

### Reentrenar el modelo PPO donde lo dejó en caso de ser necesario

In [ ]:
env = gym.make('SuperMarioBros-1-1-v0')
env = JoypadSpace(env, RIGHT_ONLY)

model3.set_env(env)

model3.learn(total_timesteps=200000, callback = callback)